# <center> <h1> 🧶 Связка `Lightning` + `ClearML` в задачах NLP. 🔤</h1> </center>

### Оглавление ноутбука
<img src='../images/nlp.webp' align="right" width="508" height="428" >
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p1">🤔 Зачем нужен Trainer?</a></li>
<li><a href="#p2">🤹‍♀️ Параметры Trainer</a></li>
<li><a href="#p7">🔫 Model, Trainer & 2 smoking CallBacks 🔫</a></li>
<li><a href="#p3">📏 Torchmetrics - удобный доступ к любым метрикам! 📐</a></li>
<li><a href="#p4">📏 TorchMetrics в PyTorch Lightning ⚡</a></li>
<li><a href="#p5">🎚 Tuner - подбираем идеальные гиперпараметры! </a></li>
<li><a href="#p6">🧸 Выводы и заключения ✅ </a></li>


    
</ul></font></p>

### 🧑‍🎓 Теперь разберем связку **PyTorch Lightning** + **ClearML**, облегчающую жизнь при работе с текстами.

<div class="alert alert-info">

**ClearML** легко интегрируется с **PyTorch Lightning**, автоматически логируя модели PyTorch, параметры, и многое другое. Все, что вам нужно сделать, это просто добавить две строки кода в ваш скрипт **PyTorch Lightning**:

<div class="alert alert-success">
    
```python
from clearml import Task

Task = Task.init(task_name="<task_name>", project_name="<project_name>")
```

<div class="alert alert-info"> 
    
Вот и всё! Это создает таск  **ClearML**, который фиксирует:
* Исходный код и несохраненные изменения
* Установленные пакеты
* Модели PyTorch
* Параметры, предоставляемые `LightningCLI`
* Всё, что мы отправляем в TensorBoard
* Весь выход консоли
* Общие сведения, такие как сведения о машине, время выполнения, дата создания и т. д.
* И многое другое

# <center id="p1">  🤔 Посмотрим на связку в деле!</center>

In [ ]:
!pip install clearml tensorboard -q

In [31]:
import os
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from lightning import (
    LightningDataModule,
    LightningModule,
    Trainer
)
from lightning.pytorch.loggers import TensorBoardLogger

from clearml import Task

os.environ['CUDA_VISIBLE_DEVICES']='0'

Вводим ключи ClearML

In [44]:
from getpass import getpass
# Введите поочерёдно полученные ключи в появившемся окне (код изменять не нужно)
access_key = getpass(prompt="Введите API Access токен: ")
secret_key = getpass(prompt="Введите API Secret токен: ")

Введите API Access токен:  ········
Введите API Secret токен:  ········


In [45]:
%%capture
#  Не показывать свои api-ключи
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

%env CLEARML_API_ACCESS_KEY=$access_key
%env CLEARML_API_SECRET_KEY=$secret_key

Для примера возьмём датасет AG NEWS, в котором содержатся новостные заметки по различным тематикам. И натренируем нейросеть определять тематику новости.

In [2]:
url = 'https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv'
news = pd.read_csv(url, names=['label', 'title', 'text'])
news.head()

,label,title,text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [3]:
news.label.value_counts()

label
3    30000
4    30000
2    30000
1    30000
Name: count, dtype: int64

In [4]:
classes = ("World", "Sports", "Business", "Sci/Tec")

## <center> Создаём Dataset и Datamodule </center>

In [5]:
class TextDataset(Dataset):
    def __init__(self, csv_file, vocab=None, tokenizer=None):
        self.data = pd.read_csv(csv_file, names=['label', 'title', 'text'])
        self.tokenizer = tokenizer or (lambda x: x.split())
        self.vocab = vocab or self.build_vocab()

    def build_vocab(self):
        '''
        Создает словарь токенов с уникальными идентификаторами, начиная с "<unk>" (0) для неизвестных токенов. 
        Метод проходит по текстам в self.data['text'], токенизирует их и добавляет новые токены с индексами, 
        равными текущему размеру словаря.

        Возвращает:
            dict: Словарь токенов с их идентификаторами.
        ''' 
    
        vocab = {"<unk>": 0}
        for text in self.data['text']:
            for token in self.tokenizer(text):
                if token not in vocab:
                    vocab[token] = len(vocab)
        return vocab

    def encode_text(self, text):
        return torch.tensor([self.vocab.get(token, self.vocab["<unk>"]) for token in self.tokenizer(text)], dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data.iloc[idx]['label'] - 1
        text = self.data.iloc[idx]['text']
        encoded_text = self.encode_text(text)
        return label, encoded_text

In [6]:
class TextDataModule(LightningDataModule):
    def __init__(self, train_csv, test_csv, batch_size=16, tokenizer=None):
        super().__init__()
        self.train_csv = train_csv
        self.test_csv = test_csv
        self.batch_size = batch_size
        self.tokenizer = tokenizer or (lambda x: x.split())

    def setup(self, stage=None):
        self.train_dataset = TextDataset(self.train_csv, tokenizer=self.tokenizer)
        self.test_dataset = TextDataset(self.test_csv, vocab=self.train_dataset.vocab, tokenizer=self.tokenizer)

        self.vocab = self.train_dataset.vocab
        self.num_classes = len(set(self.train_dataset.data['label']))

    def collate_fn(self, batch):
        labels, texts = zip(*batch)
        offsets = torch.tensor([0] + [len(text) for text in texts[:-1]]).cumsum(dim=0)
        texts = torch.cat(texts)
        labels = torch.tensor(labels, dtype=torch.long)
        return texts, offsets, labels

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, collate_fn=self.collate_fn)


## <center> Пишем класс для модели в LightningModule

В валидационный стэп добавляем промежуточное логирование текстов и предсказаний в тензорборд, чтобы они отображались в ClearML - можно в реальном времени следить как модель "умнеет", глядя на предсказания.

In [7]:
class TextSentimentModel(LightningModule):
    def __init__(self, vocab_size, embed_dim, num_class, learning_rate=1.0):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

    def training_step(self, batch, batch_idx):
        text, offsets, labels = batch
        outputs = self.forward(text, offsets)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        text, offsets, labels = batch
        outputs = self.forward(text, offsets)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(1) == labels).float().mean()

        # Log validation loss and accuracy
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

        # Log text predictions to TensorBoard
        predictions = outputs.argmax(1)
        for i in range(min(5, len(labels))):
            self.logger.experiment.add_text(
                "val_predictions",
                f'''Text: {text[i]}; 
                Prediction: {classes[predictions[i].item() - 1]}; 
                True Label: {classes[labels[i].item() - 1]}''',
                self.global_step
            )

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
        return [optimizer], [scheduler]

In [46]:
# Initialize ClearML task
task = Task.init(project_name="TextClassification", task_name="AG_NEWS Text Classification")

ClearML Task: created new task id=e668e53af0994119a75add7f9d6cd953
2025-01-21 00:00:37,713 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/651ec2987cb34c0c9613bda5583f55e9/experiments/e668e53af0994119a75add7f9d6cd953/output/log


In [37]:
# Initialize components
train_csv = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv"
test_csv = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv"
batch_size = 48
learning_rate = 0.01

In [47]:
# Сделать класс конфига и залогировать ниже
configuration_dict = {'number_of_epochs': 2, 'batch_size': 48, 'ngrams': 2, 'base_lr': 0.01}

In [48]:
# Prepare DataModule
data_module = TextDataModule(train_csv, test_csv, batch_size=batch_size)
data_module.setup()

In [49]:
vocab_size = len(data_module.vocab)
embed_dim = 32
num_class = data_module.num_classes

In [50]:
# Добавить в класс залогировать
configuration_dict = task.connect(configuration_dict)  # enabling configuration override by clearml
print(configuration_dict)  # printing actual configuration (after override in remote mode)

{'number_of_epochs': 2, 'batch_size': 48, 'ngrams': 2, 'base_lr': 0.01}


In [51]:
# Prepare Model
model = TextSentimentModel(vocab_size, embed_dim, num_class, learning_rate)

In [52]:
# Logger and Trainer
logger = TensorBoardLogger("./lightning_logs", name="text_classification")
trainer = Trainer(
    max_epochs=2,
    logger=[logger],
    accelerator="cpu",
    #devices=1
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jovyan/venvs/sinara/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [53]:
# Training
trainer.fit(model, datamodule=data_module)


  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | EmbeddingBag     | 5.0 M  | train
1 | fc        | Linear           | 132    | train
2 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
19.974    Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/venvs/sinara/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/home/jovyan/venvs/sinara/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [42]:
def predict(text, model, vocab, tokenizer):
    model.eval()
    with torch.no_grad():
        tokens = torch.tensor([vocab.get(token, vocab["<unk>"]) for token in tokenizer(text)], dtype=torch.long)
        offsets = torch.tensor([0])
        output = model(tokens, offsets)
        prediction = output.argmax(1).item()
        return prediction

In [43]:
# Load a random example from the test dataset
random_idx = torch.randint(0, len(data_module.test_dataset), (1,)).item()
example_label, example_text = data_module.test_dataset[random_idx]
predicted_label = predict(example_text, model, data_module.vocab, data_module.tokenizer)

print(f"Text: {example_text}")
print(f"True Label: {example_label}")
print(f"Predicted Label: {predicted_label}")

TypeError: Tensor.split() missing 1 required positional argument: 'split_size'

In [54]:
task.close()

В развитие:
Можно прилепить, что-то из торчметрикс, подходящее для текстов
Добавить параметров + колбэки к тренеру

## <center> Переходим в UI от ClearML и смотрим аналитику

## <center> Выводы и заключение 